# Email Scrape

Read in emails from outlook, store ones from utility EPO in dictionary, parse dictionary to get another dict of accounts, user-id, password.

## To Do

1. Scrape ngrid emails
2. Write ngrid to dict
3. Test with variety of emails
4. Pass to webscrape

### Load Modules

In [1]:
import win32com.client
import datetime
import pprint
import json

### Read Outlook Mailboxes

In [2]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
accounts= win32com.client.Dispatch("Outlook.Application").Session.Stores

mailboxes = [account.DisplayName for account in accounts]
mailboxes

['ENGIENA-GSERNASERVICES (ENGIE North America)',
 'steven.hurwitt@external.engie.com',
 'Public Folders - steven.hurwitt@external.engie.com',
 'ENGIENA-NEPOOL_DATALOADING (Engie North America)']

### Functions to Turn Messages to Dictionaries

In [41]:
def flatten(l):
    out = []
    for item in l:
        if isinstance(item, (list, tuple)):
            out.extend(flatten(item))
        else:
            if item != '' and item != ' ':
                out.append(item)
    return out

def clean_acct(acct):
    return ''.join(acct.split(' - '))

#turn date string into datetime object
def str_to_date(datestring):
    return (('date', datetime.datetime.strptime(str(datestring).split('+')[0],"%Y-%m-%d %H:%M:%S")))

def date_to_str(datetime_obj):
    return datetime.datetime.strftime(datetime_obj, format = '%m/%d/%Y %H:%M:%S')

def Merge(dict1, dict2): 
    res = {**dict1, **dict2} 
    return res 

def ba_sa_merge(a, b): 
    merged = []
    
    for k in range(0, len(a)):
        m_acct = a[k] + '_' + b[k]
        merged.append(m_acct)
    return merged


def aaron_to_dict(body):
    
    parse = body.split('\n')
    parse = [thing.splitlines() for thing in parse]

    parse2 = flatten(parse)
    strp_len = [len(item.strip(' ')) for item in parse2]

    acct_start = parse2.index('Account Number(s): ') + 1
    acct_end = strp_len.index(0)

    for item in parse2:

        split_item = item.split(': ')
        if split_item[0]  == 'User Id':
            user = ('user', split_item[1])
        
        elif split_item[0] == 'Password':
            pw = ('pw', split_item[1])
            
        elif split_item[0] == 'Customer':
            name = ('name', split_item[1])
        
        
    accts = parse2[acct_start:acct_end]
    clean_accts = [clean_acct(acct) for acct in accts]
    accts_tup = ('accts', clean_accts)
    row = [user, pw, name, accts_tup]
    return row

def admin_to_dict(body):
    
    parse = body.split('\n')
    parse = [thing.splitlines() for thing in parse]

    parse2 = flatten(parse)
    
    if (len(body) > 2500) or ("SA Number" in body):

        try:
            ba_list = [''.join(elem.split('-')) for elem in parse2[9].split(' ')]

            sa_list = [elem for elem in parse2[10].split(' ')]
            
            accts = ('accts', ba_sa_merge(ba_list, sa_list))

            for item in parse2:

                split_item = item.split(': ')
                if split_item[0]  == 'User Id':
                    user = ('user', split_item[1])
        
                elif split_item[0] == 'Password':
                    pw = ('pw', split_item[1])
            
                elif split_item[0] == 'Customer':
                    name = ('name', split_item[1])
                    
            row = [user, pw, name, accts]
                
        except:
            
            for index, item in enumerate(parse2):
                if item.lower() == 'user id':
                    special_index = index

            name = ('name', parse2[special_index + 1])
            ba_list = parse2[special_index + 2].split(' ')
            sa_list = parse2[special_index + 3].split(' ')
            user = ('user', parse2[special_index + 4])
            pw = ('pw', parse2[special_index + 6])

            ba_list = [ba.replace('-', '') for ba in ba_list]
                    
            accts = ('accts', ba_sa_merge(ba_list, sa_list))
                    
            row = [user, pw, name, accts]        
                    
                    
                    

    else:
        for item in parse2:

            split_item = item.split(': ')
        
            if split_item[0]  == 'User ID':
                user = ('user', split_item[1])
        
            elif split_item[0] == 'Password':
                pw = ('pw', split_item[1])
            
            elif split_item[0] == 'Customer':
                name = ('name', split_item[1])

            elif split_item[0] == 'Account Number':
                accts = ('accts', split_item[1])
        
        row = [user, pw, name, accts]
    
    return(row)

def ngrid_to_dict(body):
    
    parse = body.split('\n')
    parse = [thing.splitlines() for thing in parse]

    parse2 = flatten(parse)
    accts_list = []
    names_list = []
        
    for item in parse2:
        split_item = item.split(': ')
        if split_item[0] == 'Userid':
            username = split_item[1].replace(' ', '')
            user = ('user', username)
        
        if split_item[0] == 'Password':
            password = split_item[1].replace(' ', '')
            pw = ('pw', password)
        
        try:
            split_item2 = [thing.split(' ') for thing in split_item][0]
            split_item3 = list(filter(None, split_item2))
            int(split_item3[0])
            accts_list.append(split_item3[0])
            full_name = ' '.join(split_item3[1:])
            names_list.append(full_name)
        
        except:
            pass
        
    accts = ('accts', accts_list)
    names = ('names', names_list)
        
    row = [user, pw, names, accts]
    return(row)

def iter_mail(sender_func, mailbox, index):
    warning_index_log = []
    
    today = datetime.datetime.now()
    last_week = today - datetime.timedelta(7)
    last_week

    last_week_str = last_week.strftime("%Y-%m-%d %H:%M:%S")
    last_week_str2 = last_week_str +"+00:00'"

    print('scraping emails...')
    #start iterating through emails
    mail = mailbox.GetLast()
    i = index
    
    msg_row = sender_func(mail.Body)
    msg_row.append(str_to_date(mail.ReceivedTime))

    master = [(i, dict(msg_row))]
    
    i += 1
   
    while mail:
        
        mail = mailbox.GetPrevious()
        try:
            msg_row = sender_func(mail.Body)
            msg_row.append(('date', str_to_date(mail.ReceivedTime)))
            
            #if (str_to_date(mail.ReceivedTime) > last_week):
            msg_row = dict(msg_row)

            new = (i, msg_row)
        
            master.append(new)
            
            i += 1
            
        
        except:
            warning_index_log.append(index)
            
    master = dict(master)
            
    return(master, warning_index_log, i)

    

### Get Filtered Messages (Sender, Date, Body)

aaron_to_dict will gives a dictionary for each account of the structure  
  
    {'user': 'nhengi-ston3n',
         'pw': 'fm54f7',
         'name': 'STONEWALL KITCHEN LLC',
         'accts': ['800531501']}  
 
 The following code will add the date email is received (if within last week) w/ structure: 
   
     {'user': 'nhengi-ston3n',
         'pw': 'fm54f7',
         'name': 'STONEWALL KITCHEN LLC',
         'accts': ['800531501'],
         'date': datetime.datetime(2019, 5, 16, 10, 17, 21)}
 
 nest these dictionaries into a scary looking dictionary:
 
     {0: {'user': 'nhengi-ston3n', 
            'pw': 'fm54f7', 
            'name': 'STONEWALL KITCHEN LLC', 
            'accts': ['800531501'], 
            'date': datetime.datetime(2019, 5, 16, 10, 17, 21)}, 
      1: {'user': 'nhengi-morg1n', 
            'pw': '6z2s5e', 
            'name': 'MORGAN ADVANCED CERAMICS', \
            'accts': ['800514701'], 
            'date': datetime.datetime(2019, 4, 17, 12, 34, 30)}, 
      2: {'user': 'nhengi-ferr1n', 
            'pw': 'xx27b4', 
            'name': 'FERROTEC AMERICA CORP', 
            'accts': ['800511301'], 
            'date': datetime.datetime(2019, 4, 17, 12, 7, 55)},
        
            #.....

In [42]:
def get_emails():
    
    global inbox
    inbox = outlook.Folders('ENGIENA-GSERNASERVICES (ENGIE North America)')
    folder = inbox.Folders("Inbox")
    messages = folder.Items

    aaron_filter = "[SenderEmailAddress] = 'aaron.downing@eversource.com'"
    admin_filter = "[SenderEmailAddress] = 'epoadmin@eversource.com'"
    ngrid_filter = "[SenderEmailAddress] = 'michael.stanton@nationalgrid.com'"

    today = datetime.datetime.now()
    last_week = today - datetime.timedelta(days = 7)

    aaron = messages.Restrict(aaron_filter)
    admin = messages.Restrict(admin_filter)
    ngrid = messages.Restrict(ngrid_filter)
    
    print(len(aaron), 'aaron emails,', len(admin), 'epo emails', len(ngrid), 'ngrid emails')
    
    warning_index_log = []
    j = 0
    
    try:
        scrape, warnings, j2 = iter_mail(aaron_to_dict, aaron, j)
        warning_index_log.append(warnings)
        
    except:
        print('error parsing aaron')
    
    try:
        admin_scrape, warnings2, j3 = iter_mail(admin_to_dict, admin, j2)
        master = Merge(scrape, admin_scrape)
        warning_index_log.append(warnings2)
        
    except:
        print('error parsing EPO admin')
    
    try:
        ngrid_scrape, warnings3, j4 = iter_mail(ngrid_to_dict, ngrid, j3)
        master = Merge(master, ngrid_scrape)
        warning_index_log.append(warnings3)
        
    except:
        print('error parsing ngrid')
    


    print('found ', len(master.keys()), ' new emails: ')
    
    pretty_json = json.dumps(master, default = lambda date: date_to_str(date), sort_keys = True, indent = 4)
    lame_json = json.dumps(master, default = lambda date: date_to_str(date), sort_keys = True)
    
    
    print(pretty_json)
    return lame_json, master, warning_index_log
    #https://docs.microsoft.com/en-us/office/vba/api/outlook.mailitem

In [43]:
output_json, output_dict, warnings = get_emails()

221 aaron emails, 889 epo emails 278 ngrid emails
scraping emails...
scraping emails...
scraping emails...
found  1229  new emails: 
{
    "0": {
        "accts": [
            "800478502"
        ],
        "date": "05/20/2019 11:03:09",
        "name": "AMHERST LABEL INC",
        "pw": "mm59m2",
        "user": "nhengi-labl8n"
    },
    "1": {
        "accts": [
            "800531501"
        ],
        "date": [
            "date",
            "05/16/2019 10:17:21"
        ],
        "name": "STONEWALL KITCHEN LLC",
        "pw": "fm54f7",
        "user": "nhengi-ston3n"
    },
    "2": {
        "accts": [
            "800547101"
        ],
        "date": [
            "date",
            "05/16/2019 10:13:32"
        ],
        "name": "W H SILVERSTEIN, INC",
        "pw": "em55e6",
        "user": "nhengi-whsi4n"
    },
    "3": {
        "accts": [
            "800048805"
        ],
        "date": [
            "date",
            "05/16/2019 10:08:39"
        ],
        "n

In [45]:
with open('bodies.json', 'w') as f:
    json.dump(output_json, f)

## Test Area

In [25]:
global inbox
inbox = outlook.Folders('ENGIENA-GSERNASERVICES (ENGIE North America)')
folder = inbox.Folders("Inbox")
messages = folder.Items

aaron_filter = "[SenderEmailAddress] = 'aaron.downing@eversource.com'"
admin_filter = "[SenderEmailAddress] = 'epoadmin@eversource.com'"
ngrid_filter = "[SenderEmailAddress] = 'michael.stanton@nationalgrid.com'"


aaron = messages.Restrict(aaron_filter)
admin = messages.Restrict(admin_filter)
ngrid = messages.Restrict(ngrid_filter)

test, warn, j = iter_mail(aaron_to_dict, aaron, 0)

test2, warn2, n = iter_mail(admin_to_dict, admin, 0)

    
test3, warn3, m = iter_mail(ngrid_to_dict, ngrid, 0)

scraping emails...
scraping emails...
scraping emails...


In [12]:
test

[(0,
  {'user': 'nhengi-labl8n',
   'pw': 'mm59m2',
   'name': 'AMHERST LABEL INC',
   'accts': ['800478502'],
   'date': datetime.datetime(2019, 5, 20, 11, 3, 9)})]

In [13]:
test2

NameError: name 'test2' is not defined

In [14]:
test3

[(0,
  {'user': 'SUEZ_HIST',
   'pw': '3824',
   'names': ['CASEY KEMP'],
   'accts': ['1538196030'],
   'date': datetime.datetime(2019, 5, 21, 15, 49, 5)})]

In [36]:
item = admin.GetPrevious().Body
print(item)

Hello Engie Resources,

 

The NEWTON MA OWNER LLC EPO Online account is now operational and ready for immediate use.  The start of the one-month service is today, 05/21/2019 and will end 06/22/2019.  Below is the user id and password to begin accessing the data immediately.  

 

Customer: NEWTON MA OWNER LLC

Account Number: 26532251050

 

Issue # 0

 

User ID: newton1-nengie

Password: xr6911

 

Please note, the user id and password are case-sensitive.

 

*** Accounts not listed do not have interval data available. ***

 

To access this account online, click on this link...

http://eversource.epo.schneider-electric.com/eversource <https://urldefense.proofpoint.com/v2/url?u=http-3A__eversource.epo.schneider-2Delectric.com_eversource&d=CwMBAg&c=Rt9MH7x8aPAwEY3f-URIJch7v0PDyVhHmVdpquKSoc0&r=ln6617U_vDodf1tBqDzdh2Xvhdbj376DpMyBhO_kzE8&m=1LlJO4dOFCJibG8r4Ks3nQC50q_2WqegCw4paPJrY1w&s=WB8Tnc-ZbYFNgrhMbwle3R4Bl-C70RZmdTbKt3fN9co&e=> 

 

If you have any questions, please contact EPOAdm

In [24]:
item = admin.GetPrevious().Body
#parse = item.split('\r')
#parse = [thing.splitlines() for thing in parse]

#parse2 = flatten(parse)
for index, item in enumerate(parse2):
    if item.lower() == 'user id':
        special_index = index

name = parse2[special_index + 1]
ba_list = parse2[special_index + 2].split(' ')
sa_list = parse2[special_index + 3].split(' ')
user = parse2[special_index + 4]
pw = parse2[special_index + 6]

ba_list = [ba.replace('-', '') for ba in ba_list]
ba_sa = ba_sa_merge(ba_list, sa_list)

print(name)
print(ba_sa)
print(user)
print(pw)

Griffin Industrial Realty
['51855942025_282371006']
griff-engie
782cc


In [18]:
test.append(test3)

In [40]:
test

[(0,
  {'user': 'nhengi-labl8n',
   'pw': 'mm59m2',
   'name': 'AMHERST LABEL INC',
   'accts': ['800478502'],
   'date': datetime.datetime(2019, 5, 20, 11, 3, 9)}),
 [(0,
   {'user': 'newton1-nengie',
    'pw': 'xr6911',
    'name': 'NEWTON MA OWNER LLC',
    'accts': '26532251050',
    'date': datetime.datetime(2019, 5, 21, 10, 48, 39)})],
 [(0,
   {'user': 'newton1-nengie',
    'pw': 'xr6911',
    'name': 'NEWTON MA OWNER LLC',
    'accts': '26532251050',
    'date': datetime.datetime(2019, 5, 21, 10, 48, 39)})],
 [(0,
   {'user': 'SUEZ_HIST',
    'pw': '3824',
    'names': ['TOWN OF NORTH KINGST'],
    'accts': ['0336183003'],
    'date': datetime.datetime(2019, 5, 17, 15, 32)})]]